In [30]:
import gym
import simple_driving
#import pybullet_envs
import pybullet as p
import matplotlib.pyplot as plt
from IPython import display as ipythondisplay
from pyvirtualdisplay import Display
from IPython.display import HTML
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
import math
from collections import defaultdict
import pickle
from IPython.display import clear_output
import torch
import random

#display = Display(visible=0, size=(400, 300))
#display.start()

def display_video(frames, framerate=30):
  """Generates video from `frames`.

  Args:
    frames (ndarray): Array of shape (n_frames, height, width, 3).
    framerate (int): Frame rate in units of Hz.

  Returns:
    Display object.
  """
  height, width, _ = frames[0].shape
  dpi = 70
  orig_backend = matplotlib.get_backend()
  matplotlib.use('Agg')  # Switch to headless 'Agg' to inhibit figure rendering.
  fig, ax = plt.subplots(1, 1, figsize=(width / dpi, height / dpi), dpi=dpi)
  matplotlib.use(orig_backend)  # Switch back to the original backend.
  ax.set_axis_off()
  ax.set_aspect('equal')
  ax.set_position([0, 0, 1, 1])
  im = ax.imshow(frames[0])
  def update(frame):
    im.set_data(frame)
    return [im]
  interval = 1000/framerate
  anim = animation.FuncAnimation(fig=fig, func=update, frames=frames,
                                  interval=interval, blit=True, repeat=False)
  return HTML(anim.to_html5_video())

In [31]:
def epsilon_greedy(env, state, episodes, episode, model):
    """Selects an action to take based on a uniformly random sampled number. 
    If this number is greater than epsilon then returns action with the largest
    Q-value at the current state. Otherwise it returns a random action. This
    version decays epsilon from a large number to a small number over the duration
    of training. This results in highly likely random actions at the start and
    eventually biasing actions towards those with high q-values towards the end.

    Args:
        env: gym object.
        state: current state
        episodes: maximum number of episodes
        episode: number of episodes played so far
        model: Q-function approximator

    Returns:
        Action to be executed for next step.
    """
    EPS_START = 0.99
    EPS_END = 0.15
    EPS_DECAY = episodes
    sample = np.random.uniform(0, 1)
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        math.exp(-1. * episode / EPS_DECAY)
    if sample > eps_threshold:
        q_values = model(torch.tensor([state], dtype=torch.float32))    # predict q-values for state
        return q_values.argmax().item()                                 # return action with highest q-value
    else:
        return np.random.choice(np.array(range(8)))  # incorporate prior here to prevent flapping too much during exploration (agent will always keep trying to fly into the sky otherwise)
        #return np.random.choice([6,7,8])

def simulate(env, max_episode_length, episodes, episode, model):
    """Rolls out an episode of actions to be used for learning.

    Args:
        env: gym object.
        episodes: maximum number of episodes
        episode: number of episodes played so far

    Returns:
        Dataset of episodes for training the RL agent containing states, actions and rewards.
    """
    D = []
    state = env.reset()                                                   # line 2
    done = False
    env.getExtendedObservation()
    prev_reward = 0
    #while True:                                                             # here I chose to not let episode end but you could replace with the line below
    for step in range(max_episode_length):                                  # line 3
        action = epsilon_greedy(env, state, episodes, episode, model)  # line 4
        next_state, reward, done,info = env.step(action)                   # line 5

        D.append([state, action, reward, next_state])                       # line 7
        state = next_state                                                  # line 8
        if done:                                                            # if we fall into
            break
    return D                                                                # line 10

def approx_q_learning(env, gamma, episodes, max_episode_length, model, loss_fn, optimizer):
    """Main loop of Approximate Q-learning algorithm.

    Args:
        env: gym object.
        gamma: discount factor - determines how much to value future actions
        episodes: number of episodes to play out
        max_episode_length: maximum number of steps for episode roll out
        model: Q-function approximator
        loss_fn: the loss function for our function approximator
        optimizer: for backpropagating the gradient of our loss

    Returns:
        Q-function which is used to derive policy.
    """
    total_reward = 0
    for episode in range(episodes):                                             # slightly different to line 3, we just run until maximum episodes played out
        D = simulate(env, max_episode_length, episodes, episode, model)    # line 4
        q_values_batch = []
        target_batch = []
        for data in D:                                                          # data = [state, action, reward, next_state]  (line 5)
            ####################### update Q values (line 6-7) #########################
            state  = data[0]
            action = data[1]
            reward = data[2]
            #print(f"this is reward = {reward}")
            next_state = data[3]
            q_values = model( torch.tensor([state], dtype=torch.float32) )            # predict Q-value for current state
            q_values_next = model(torch.tensor([next_state], dtype=torch.float32))  # predict Q-value for next state
            target = q_values.clone().detach()
            target[0][action] = reward + gamma * q_values_next.max().item()         # loss between prediction and true Q-value we just found from interacting with the env
            total_reward += data[2]
            loss = loss_fn(q_values, target)                                        # compute loss
            optimizer.zero_grad()
            loss.backward()                                                         # backpropogate the loss (compute gradients)
            optimizer.step()                                                        # update model using gradients
            #########################################################################    
        if episode % 100 == 0:
            print("average total reward per episode batch since episode ", episode, ": ", total_reward/ float(100))
            total_reward = 0
    return model  # line 9

In [32]:
######################### renders image from third person perspective for validating policy ##############################
#env = gym.make("SimpleDriving-v0", apply_api_compatibility=True, renders=False, isDiscrete=True, render_mode='tp_camera') 
##########################################################################################################################

######################### renders image from onboard camera ###############################################################
# env = gym.make("SimpleDriving-v0", apply_api_compatibility=True, renders=False, isDiscrete=True, render_mode='fp_camera') 
##########################################################################################################################

######################### if running locally you can just render the environment in pybullet's GUI #######################
#env = gym.make("SimpleDriving-v0", apply_api_compatibility=True, renders=False, isDiscrete=True,render_mode='fp_camera') 
##########################################################################################################################

env = gym.make("SimpleDriving-v0", apply_api_compatibility=False, isDiscrete = True) 

state = env.reset()



# Neural network params
l1 = 4   # this is set as two as we are just inputting x and y , this environment has more available inputs                    
l2 = 32                         # hidden layer dimension (we choose this)
l3 = env.action_space.n                         # output layer dimension - same as action space
alpha = 0.0001                    # learning rate for updating the neural network weights

# assemble neural network based on above params 
model = torch.nn.Sequential(
    torch.nn.Linear(l1, l2),
    torch.nn.LeakyReLU(),
    torch.nn.Linear(l2, l3)
)

loss_fn = torch.nn.MSELoss()                                # mean square error loss function
optimizer = torch.optim.Adam(model.parameters(), lr=alpha)  # optimizer for updating neural network weights


gamma = 0.4              # discount factor - determines how much to value future actions
episodes = 7001           # number of episodes to play out
max_episode_length = 10000

approx_q_learning(env, gamma, episodes, max_episode_length, model, loss_fn, optimizer)
env.close()

average total reward per episode batch since episode  0 :  -1.6799673772650954
average total reward per episode batch since episode  100 :  -107.30273432748776


KeyboardInterrupt: 

In [ ]:
import time
env = gym.make("SimpleDriving-v0", apply_api_compatibility=True, renders=True, isDiscrete=True,render_mode = 'fp_camera')
state, info = env.reset()

for i in range(10000):
    q_values = model(torch.tensor([state], dtype=torch.float32))  # predict q-values using learnt model
    action = q_values.argmax().item()                             # select action with highest predicted q-value
    state, reward, done,_, info = env.step(action)
    time.sleep(0.05)
    print(reward)
    if done:
        break

env.close()

0.6565967505576269
0.6720379062009868
0.7457403663612363
0.563736263531227
0.631991508787218
0.44376602484689953
0.5104164591871845
0.5740854469435681
0.6347561015160595
0.4492726982656233
0.5106666282719814
0.5692224238436714
0.6247338805224638
0.6769904804805791
0.7023612085009137
0.6302228550336872
0.5572017659779056
0.4835686304168486
0.6315435745106955
0.5579379522061332
0.6634239308155383
0.5478164027653465
0.4283558608411139
0.35173719130695236
0.2731808682370026
0.1927909873836322
0.11067608816985608
0.026948906350910518
-0.05827388855339888
-0.1448719692285937
-0.23272163844283977
-0.13132829486102726
-0.03849851894686829
0.04483796546334684
-0.07563406437519804
0.003363730134708698
-0.11489726771058073
-0.040408300895440674
-0.1894030287121794
-0.34861578388494285
-0.5171976987999312
-0.5038478361010879
-0.6797427992680358
-0.666401425285597
-0.6710024246843043
-0.6859358711202346
-0.7070153490983246
-0.732329355428615
-0.7611592558027953
-0.7154554961718245
-0.66019112731821